**Programming assignment:**\
An electron passes through the plates of an electron gun, entering with a speed $v_0 = 10^6$ m/s. The distance between the plates is  $H = 1\mu$m. The voltage difference between plates is $\Delta V = 1\mu$V. We are interested to calculate the travel time of the electron between plates. The electric field between plates is calculate with the formula $E = \Delta V/H$, and therefore the electron moves with an acceleration given by $a = q E/m_e$, where $q$ and $m_e$ are the charge and the mass of the electron, respectively. The kinematic equation to be solve is therefore $$ \frac 12 a t^2 + v_0 t = H$$

This is a quadratic equation of the type $a x^2 + b x + c = 0$, for which we know the standard formula

$$x = \frac{-b \pm \sqrt{b^2 - 4 ac}}{2a}$$

To solve this problem, you need to write a Python function that solves a general quadratic equation specified by parameters $a$, $b$, and $c$. When applied to the present problem, the naive implementation of the standard quadratic formula should fail because of catastrophic loss of precision. Show explicitly how the results for the numbers at hand differ from your expectations. To get an expectation you may need to use the useful approximation $\sqrt{1 + \epsilon} \approx 1 + \epsilon/2$

To fix this deficiency, implement a second function that uses the alternate quadratic equation formula

$$
x = \frac{2c}{-b \pm \sqrt{b^2 - 4 ac}}
$$

First prove that this is true, and then demonstrate that it works even in the extreme situation of this problem.

------

### Solution - part 1

Between the two roots of the quadratic equation the root

$$x = \frac{-b + \sqrt{b^2 - 4 ac}}{2a}$$

is problematic because it may happen that $4ac \ll b^2$ and then $\sqrt{b^2 - 4 ac} \approx b$ and we have a difference between numbers that are nearly equals giving wrongly that $x = 0$.  On the other hand, we can check that 

$$x = \frac{2c}{-b - \sqrt{b^2 - 4 ac}}$$ 

does not suffer from this round-off problems and is mathematically the same. We can verify that

$$\frac{-b + \sqrt{b^2 - 4 ac}}{2a} = \frac{2c}{-b - \sqrt{b^2 - 4 ac}}$$ because

$$ 4 a c = (-b + \sqrt{b^2 - 4 ac}) (-b - \sqrt{b^2 - 4 ac}) = - \left[ (b^2 - 4 a c) - b^2\right] = 4ac$$

When $b$ is negative, the other root may fault, and corresponds to the other alternate root.

The best way to compute the roots is to first calculate

$$ q = -\frac 12 \left[ b + \mbox{sgn}(b) \sqrt{b^2 - 4 ac} \right] $$

and get $x_1 = q/a$ and $x_2 = c/q$. This is quick since square root is called only once, and also stable against round-off.

Let's try on these ideas

In [3]:
import math
def sol_bad(a, b, c):
    x1 = (-b + math.sqrt(b**2 - 4*a*c))/(2*a)
    x2 = (-b - math.sqrt(b**2 - 4*a*c))/(2*a)
    return((x1, x2))

def signum(x):
    return (x > 0) - (x < 0)

def sol_good(a, b, c):
    q = -0.5*(b + signum(b)*math.sqrt(b**2 - 4*a*c))
    x1 = q/a
    x2 = c/q
    return((x1, x2))

First on something easy:

In [4]:
a, b, c = 1.0, -1.0, -6.0
print(sol_bad(a, b, c), " =?= ", sol_good(a, b, c))

(3.0, -2.0)  =?=  (3.0, -2.0)


then for a difficult situation

In [6]:
a, b, c = 0.002, 200_000., -0.003
print(sol_bad(a, b, c), " =?= ", sol_good(a, b, c))

(1.4551915228366852e-08, -100000000.00000001)  =?=  (-100000000.00000001, 1.5e-08)


and, now brace for disaster

In [7]:
a, b, c = 0.002, 300_000., -0.002
print(sol_bad(a, b, c), " =?= ", sol_good(a, b, c))

(0.0, -150000000.0)  =?=  (-150000000.0, 6.666666666666667e-09)


Hmm, ... I am starting to have doubts even about the so-called "good" solution. How I can be sure it is correct?
The disaster came because this is really a corner case when $ 4 a c \ll b^2$. In this case we can use the approximaton $\sqrt{1 + \epsilon} \approx 1 + \epsilon/2$, good for very small $\epsilon \ll 1$, in this way:

$$ x_1 = \frac{-b + \sqrt{b^2 - 4 ac}}{2a} = \frac{-b + b \sqrt{1 - 4 ac/b^2}}{2a}
\approx \frac{ -b + b (1 - 2 ac/b^2)}{2a} = -\frac cb$$

So the worst case $x_1 = - 0.002/300,000 =  6.666666666666667e-09$. So, yes, it works perfectly this time. Check it:

In [100]:
-c/b

6.666666666666667e-09

### Solution - part 2

Now we can solve the physics problem at hand with some confidence.
We first lay out the known quantities:

In [11]:
q = 1.602176634e-19
m = 9.10938356e-31
v0 = 2.e8
H = 1.e-9
ΔV = 1.e-9

... calculate the coefficients for the quadratic equation for traversal time $a t^2/2 + v_0 t - H = 0$

In [12]:
E = ΔV/H
acc = q*E/m
a = 0.5*acc
b = v0
c = -H

... and use the solvers (bad and good) to get a solution

In [13]:
print(sol_bad(a, b, c))
print(sol_good(a, b, c))

(5.083349360747667e-18, -0.0022742520060993524)
(-0.0022742520060993524, 4.999999999999989e-18)


The negative solution is not of interest. We can see differences starting to appear in the positive solution after 8 digits after the decimal point. So is not that bad yet. But if velocity is close to the speed of light, and the distance decrease to 1 nm and voltage to 1 nV, the result has only one digit of precision!

What is the expectation for time in these extreme cases?

$t \approx - c/b = H/v_0$

Charge, mass, electric field, are irrelevant, for such short times and large speeds the particle moves freely.